In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# TrailMake A

In [2]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\TrailMakeA
data = pd.read_csv('TrailMakeA_with_MethylomeDetails_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\TrailMakeA


,Unnamed: 0,Sentrix,PATNO,Gender,Age,AgeCate,HYS,APPRDX,Trail_Making_A,cg08012045,...,cg00956423,cg14462343,cg07481734,cg22394535,cg13044444,cg23223756,cg16870358,cg04754776,cg00154746,cg06155771
0,1,200991620021_R01C01,3001,1,65.1,4,2,1,1.0,0.962437,...,0.10987,0.056582,0.064137,0.053802,0.083556,0.042638,0.041733,0.896521,0.033285,0.753096


In [3]:
X = data.drop(['Unnamed: 0', 'Sentrix','PATNO','HYS','Gender','Age','AgeCate','APPRDX','Trail_Making_A'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg08012045  cg08300930  cg00134295  cg10537824  cg04990650  cg03379001  \
0    0.962437    0.964442    0.243530    0.826172    0.845330    0.852316   
1    0.938567    0.960982    0.207738    0.792745    0.796766    0.838331   

   cg02215627  cg06699201  cg18100935  cg19220149  ...  cg00956423  \
0    0.871498    0.653140    0.891670    0.874343  ...     0.10987   
1    0.857896    0.816283    0.897772    0.833252  ...     0.13650   

   cg14462343  cg07481734  cg22394535  cg13044444  cg23223756  cg16870358  \
0    0.056582    0.064137    0.053802    0.083556    0.042638    0.041733   
1    0.038891    0.039595    0.026053    0.051206    0.029660    0.027424   

   cg04754776  cg00154746  cg06155771  
0    0.896521    0.033285    0.753096  
1    0.868682    0.020491    0.888800  

[2 rows x 87718 columns]


In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6875

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	87718


BorutaPy(estimator=RandomForestClassifier(n_estimators=4188,
                                          random_state=RandomState(MT19937) at 0x1AC060C1E40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1AC060C1E40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
52196,cg10623221,2
14887,cg04001014,2
5585,ch.8.2598327F,3


In [10]:
Top50.to_csv ('TrailMakeA_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('TrailMakeA_Boruta_All_Male.csv', index = False)

In [11]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('TrailMakeA_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11008\2872064585.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg23328194,ch.8.2598327F,cg03584202,cg05720640,cg16108246,cg03060781,cg05955802,cg04001014,cg21608703,cg01513659,...,cg09159198,cg03619568,cg16912215,cg02180305,cg15043720,cg16933706,cg17652025,cg26819609,cg07880064,APPRDX
0,0.722162,0.036232,0.914221,0.345524,0.074928,0.823503,0.035858,0.038820,0.049725,0.826730,...,0.652014,0.836325,0.536352,0.640409,0.057796,0.077437,0.042378,0.118916,0.797291,1
1,0.728596,0.024407,0.919825,0.276704,0.083055,0.901964,0.019171,0.033699,0.041892,0.776323,...,0.771486,0.629661,0.384349,0.647882,0.027904,0.031536,0.042088,0.077118,0.764450,2


# Dream

In [12]:
%cd Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\Dream
data = pd.read_csv('Dream_with_MethylomeDetails_Male.csv')
data.head(1)

Z:\PPMI_Data\Excels\NonMotors\The251\Male\GeneCpG\Dream


,Unnamed: 0,Sentrix,PATNO,Gender,Age,AgeCate,HYS,APPRDX,REM_Dream,cg07920064,...,cg12444710,cg01442833,cg17429457,cg26703873,cg21642591,cg06818627,cg20495247,cg12696399,cg22978941,cg12009719
0,3,200973410167_R05C01,3010,1,47.0,2,2,1,2,0.020416,...,0.875451,0.022926,0.863662,0.907782,0.022595,0.019975,0.046285,0.021058,0.031956,0.590721


In [13]:
X = data.drop(['Unnamed: 0', 'Sentrix','PATNO','HYS','Gender','REM_Dream','Age','AgeCate','APPRDX'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   cg07920064  cg27486695  cg08290072  cg19964303  cg24775454  cg26544277  \
0    0.020416    0.010285    0.028790    0.967632    0.034748    0.021628   
1    0.011673    0.007561    0.014732    0.977709    0.018826    0.010112   

   cg23153680  cg06716437  cg18014247  cg19412109  ...  cg12444710  \
0    0.106042    0.188028    0.060150    0.134090  ...    0.875451   
1    0.028173    0.123297    0.040974    0.136673  ...    0.857053   

   cg01442833  cg17429457  cg26703873  cg21642591  cg06818627  cg20495247  \
0    0.022926    0.863662    0.907782    0.022595    0.019975    0.046285   
1    0.023881    0.829684    0.916199    0.039999    0.031587    0.041728   

   cg12696399  cg22978941  cg12009719  
0    0.021058    0.031956    0.590721  
1    0.034263    0.026730    0.514764  

[2 rows x 639 columns]


In [14]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [15]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.6363636363636364

In [16]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100


BorutaPy finished running.

Iteration: 	23 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	639


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x1AC40545440),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1AC40545440, verbose=1)

In [17]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [18]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [19]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
607,cg25745663,2
113,cg05519181,3
466,cg27068143,4


In [20]:
Top50.to_csv ('Dream_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('Dream_Boruta_All_Male.csv', index = False)

In [21]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['PATNO'] = data['PATNO']
df_subset['APPRDX'] = data['APPRDX']
df_subset.to_csv ('Dream_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_11008\1028601944.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['PATNO'] = data['PATNO']
C:\Users\system 4\AppData\Local\Temp\ipykernel_11008\1028601944.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['APPRDX'] = data['APPRDX']


,cg25836061,cg12175949,cg13463109,cg04692870,cg05519181,cg07795168,cg25935985,cg03129541,cg24096745,cg00744924,...,cg10365880,cg07308881,cg09132121,cg20878697,cg14276366,cg25745663,cg18016466,cg19258682,PATNO,APPRDX
0,0.908477,0.019751,0.873580,0.565374,0.872761,0.977116,0.210510,0.012383,0.905673,0.879814,...,0.924827,0.699845,0.889340,0.780733,0.915085,0.016247,0.020425,0.021563,3010,1
1,0.896387,0.012827,0.857712,0.663156,0.836375,0.980372,0.190329,0.007813,0.946070,0.907005,...,0.940162,0.763235,0.929361,0.876878,0.859082,0.011207,0.014503,0.011485,3024,1
